europeantour.com has everything but not easily scrapable
https://www.europeantour.com/european-tour/masters-tournament-1990/results?round=4

In [1]:
import csv, re, time
import numpy as np
import pandas as pd
from scipy.stats import kendalltau, iqr

In [2]:
import mechanize
from selenium import webdriver

In [3]:
br = mechanize.Browser()

br.set_handle_robots(False)
br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]


## The Open

In [4]:
response = br.open('https://www.theopen.com/previous-opens')

In [5]:
last_ten_opens_links = ['https://www.theopen.com'+l.url for l in br.links() if 'previous-opens/1' in l.url]

#last_ten_opens_links

In [6]:
#Can't easily access the links for 1990-2009 - easiest to add them manually
prev_twenty_opens_links = ['138th-open-turnberry-2009',
                           '137th-open-royal-birkdale-2008',
                           '136th-open-carnoustie-2007',
                           '135th-open-royal-liverpool-2006',
                           '134th-open-st-andrews-2005',
                           '133rd-open-royal-troon-2004',
                           '132nd-open-royal-st-georges-2003',
                           '131st-open-muirfield-2002',
                           '130th-open-royal-lytham-st-annes-2001',
                           '129th-open-st-andrews-2000',
                           '128th-open-carnoustie-1999',
                           '127th-open-royal-birkdale-1998',
                           '126th-open-royal-troon-1997',
                           '125th-open-royal-lytham-st-annes-1996',
                           '124th-open-st-andrews-1995',
                           '123rd-open-turnberry-1994',
                           '122nd-open-royal-st-georges-1993',
                           '121st-open-muirfield-1992',
                           '120th-open-royal-birkdale-1991',
                           '199th-open-st-andrews-1990'] #this link is named wrongly on the site
prev_twenty_opens_links = ['https://www.theopen.com/previous-opens/'+e+'/' for e in prev_twenty_opens_links]

In [7]:
last_thirty_opens_links = last_ten_opens_links + prev_twenty_opens_links

In [8]:
response = br.open('125th-open-royal-lytham-st-annes-1996')
pagetext = response.readlines()
pagetext = [b.decode() for b in pagetext]

In [35]:
theopen_leaderboards_dict = {}

for l in last_thirty_opens_links[-1::-1]:
    year = l.split('/')[-2].split('-')[-1]
    
    response = br.open(l)
    pagetext = response.readlines()
    pagetext = [b.decode() for b in pagetext]
        
    leaderboard = []
    scores = []
    for row in pagetext:
        if 'leaderboard__content-name' in row or 'final-leaderboard__cell--par' in row:
            if scores:
                if len(scores) == 5:
                    leaderboard.append({'player': player, 'R1': int(scores[1]), 'R2': int(scores[2]), 'R3': int(scores[3]), 'R4': int(scores[4])})
                elif len(scores) == 4:
                    leaderboard.append({'player': player, 'R1': int(scores[0]), 'R2': int(scores[1]), 'R3': int(scores[2]), 'R4': int(scores[3])})
                elif len(scores) == 2:
                    leaderboard.append({'player': player, 'R1': int(scores[0]), 'R2': int(scores[1]), 'R3': None, 'R4': None})
                scores = []

            if 'leaderboard__content-name' in row:
                player = re.findall('content-name">([^<]*)<', row)[0]
                #print(player)
                scores = []
        number = re.findall('<div class="final-leaderboard__content">(\d{2})</div>', row)
        if len(number) > 0:
            #print(number[0])
            scores.append(number[0])
    
    print(year, len(leaderboard))
    
    leaderboard_df = pd.DataFrame(leaderboard)
    leaderboard_df['made_cut'] = leaderboard_df['R3'].notnull() & leaderboard_df['R4'].notnull()
    leaderboard_df['total_at_R1'] = leaderboard_df['R1']
    leaderboard_df['total_at_R2'] = leaderboard_df['R1'] + leaderboard_df['R2']
    leaderboard_df['total_at_R3'] = leaderboard_df['total_at_R2'] + leaderboard_df['R3']
    leaderboard_df['total_at_R4'] = leaderboard_df['total_at_R3'] + leaderboard_df['R4']
    leaderboard_df['pos_at_R1'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R1 < row['total_at_R1']) + 1, axis=1)
    leaderboard_df['pos_at_R2'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R2 < row['total_at_R2']) + 1, axis=1)
    leaderboard_df['pos_at_R3'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R3 < row['total_at_R3']) + 1, axis=1)
    leaderboard_df['pos_at_R4'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R4 < row['total_at_R4']) + 1, axis=1)

    theopen_leaderboards_dict[int(year)] = leaderboard_df

1990 156
1991 155
1992 156
1993 156
1994 156
1995 157
1996 153
1997 152
1998 153
1999 154
2000 156
2001 154
2002 153
2003 152
2004 156
2005 155
2006 155
2007 156
2008 154
2009 155
2010 156
2011 155
2012 155
2013 153
2014 155
2015 156
2016 154
2017 156
2018 155
2019 156


In [37]:
driver = webdriver.Firefox()

In [38]:
#Add 2021; there was none in 2020

driver.get('https://www.theopen.com/leaderboard')
time.sleep(2)

if driver.find_elements_by_class_name('cookie-warning__agree-button')[0].is_displayed():
    driver.find_elements_by_class_name('cookie-warning__agree-button')[0].click() #clear the cookies pop-up
    time.sleep(1)
driver.find_elements_by_class_name('leaderboard__cell-r-score-container')[0].click() #show the round scores
time.sleep(1)

leaderboard = []
for row in driver.find_elements_by_class_name('leaderboard__row'):
    if 'CHAMPION' in row.find_elements_by_class_name('leaderboard__cell.leaderboard__cell--player')[0].text:
        player_name = 'Collin MORIKAWA'
    else:
        player_name = row.find_elements_by_class_name('leaderboard__cell.leaderboard__cell--player')[0]\
            .find_elements_by_class_name('leaderboard__cell-player-firstname')[0].text + ' ' + row.find_elements_by_class_name('leaderboard__cell-player-lastname')[0].text
    scores = row.find_elements_by_class_name('leaderboard__cell.leaderboard__cell--round')
    scores = [int(s.text) if s.text != '-' else None for s in scores]
    #print(player_name, scores)
    leaderboard.append({'player': player_name, 'R1': scores[0], 'R2': scores[1], 'R3': scores[2], 'R4': scores[3]})
    
leaderboard_df = pd.DataFrame(leaderboard)
leaderboard_df['made_cut'] = leaderboard_df['R3'].notnull() & leaderboard_df['R4'].notnull()
leaderboard_df['total_at_R1'] = leaderboard_df['R1']
leaderboard_df['total_at_R2'] = leaderboard_df['R1'] + leaderboard_df['R2']
leaderboard_df['total_at_R3'] = leaderboard_df['total_at_R2'] + leaderboard_df['R3']
leaderboard_df['total_at_R4'] = leaderboard_df['total_at_R3'] + leaderboard_df['R4']
leaderboard_df['pos_at_R1'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R1 < row['total_at_R1']) + 1, axis=1)
leaderboard_df['pos_at_R2'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R2 < row['total_at_R2']) + 1, axis=1)
leaderboard_df['pos_at_R3'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R3 < row['total_at_R3']) + 1, axis=1)
leaderboard_df['pos_at_R4'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R4 < row['total_at_R4']) + 1, axis=1)

theopen_leaderboards_dict[2021] = leaderboard_df

In [39]:
#kts_r12, kts_r23, kts_r34 = [], [], []
mapc_r12, mapc_r23, mapc_r34 = [], [], []

for year, this_l_df in theopen_leaderboards_dict.items():
    #print('\n'+year+'\n')
    
    #Compare two ordered lists
    order_at_R1 = this_l_df.sort_values('total_at_R1').player.tolist()
    order_at_R2 = this_l_df.sort_values('total_at_R2').player.tolist()
    order_at_R1_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R1').player.tolist()
    order_at_R2_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R2').player.tolist()
    order_at_R3_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R3').player.tolist()
    order_at_R4_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R4').player.tolist()
    #print(f"{len(order_at_R1)} in R1 list, {len(order_at_R2)} in R2 list, {len(order_at_R2_tophalf)} in R2 cut only list, {len(order_at_R3_tophalf)} in R3 list, {len(order_at_R4_tophalf)} in R4 list")
    
    #print(f"{np.mean([a == b for a,b in zip(order_at_R1, order_at_R2)])*100:.1f}% entries in rankings stayed same R1 to R2 (full)")
    #print(f"{np.mean([a == b for a,b in zip(order_at_R2_tophalf, order_at_R3_tophalf)])*100:.1f}% entries in rankings stayed same R2 to R3")
    #print(f"{np.mean([a == b for a,b in zip(order_at_R3_tophalf, order_at_R4_tophalf)])*100:.1f}% entries in rankings stayed same R3 to R4")
    
    mapc_r12.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R1 - this_l_df[this_l_df.made_cut].pos_at_R2)))
    #print(f"Mean abs pos change R1->R2 for top half = {mapc_r12[-1]:.1f}")
    mapc_r23.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R2 - this_l_df[this_l_df.made_cut].pos_at_R3)))
    #print(f"Mean abs pos change R2->R3 for top half = {mapc_r23[-1]:.1f}")
    mapc_r34.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R3 - this_l_df[this_l_df.made_cut].pos_at_R4)))
    #print(f"Mean abs pos change R3->R4 for top half = {mapc_r34[-1]:.1f}")
    
#print(f"\nMean Kendall-Taus for R1->R2, R2->R3, R3->R4 for top half are {np.mean(kts_r12):.3f}, {np.mean(kts_r23):.03f}, {np.mean(kts_r34):.03f}")
print(f"Mean abs pos changes for top half are {np.mean(mapc_r12):.1f}, {np.mean(mapc_r23):.1f}, {np.mean(mapc_r34):.1f}")

Mean abs pos changes for top half are 22.3, 14.4, 11.3


In [40]:
#Are low scores more likely in any round?
#Selecting the top half like this will bias R1 and R2 towards low scores, because those players had to play quite well in R1,2 but not in R3,4
#R3 and R4 look similar; if anything R4 is lower

print(f"Mean 10% score in R1 for top half = {np.mean([df[df.made_cut].R1.quantile(0.10) for df in theopen_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R2 for top half = {np.mean([df[df.made_cut].R2.quantile(0.10) for df in theopen_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R3 for top half = {np.mean([df[df.made_cut].R3.quantile(0.10) for df in theopen_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R4 for top half = {np.mean([df[df.made_cut].R4.quantile(0.10) for df in theopen_leaderboards_dict.values()]):.1f}")

Mean 10% score in R1 for top half = 68.2
Mean 10% score in R2 for top half = 68.4
Mean 10% score in R3 for top half = 68.8
Mean 10% score in R4 for top half = 68.6


In [41]:
#Do scores become more stretched in R3?
#Not sure if made cut gives any bias to R1,2 here.

#IQR seems highest in R4, then R3 - more going for broke?

print(f"Mean round score IQR in R1 for top half = {np.mean([iqr(df[df.made_cut].R1) for df in theopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R2 for top half = {np.mean([iqr(df[df.made_cut].R2) for df in theopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R3 for top half = {np.mean([iqr(df[df.made_cut].R3) for df in theopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R4 for top half = {np.mean([iqr(df[df.made_cut].R4) for df in theopen_leaderboards_dict.values()]):.1f}")

Mean round score IQR in R1 for top half = 3.1
Mean round score IQR in R2 for top half = 2.9
Mean round score IQR in R3 for top half = 3.8
Mean round score IQR in R4 for top half = 3.8


In [42]:
#Do total scores become more stretched by R3?
#Not sure if made cut gives any bias to R1,2 here.

#Total score IQR increases by round, so it will be harder to move positions.

print(f"Mean round score IQR in R1 for top half = {np.mean([iqr(df[df.made_cut].total_at_R1) for df in theopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R2 for top half = {np.mean([iqr(df[df.made_cut].total_at_R2) for df in theopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R3 for top half = {np.mean([iqr(df[df.made_cut].total_at_R3) for df in theopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R4 for top half = {np.mean([iqr(df[df.made_cut].total_at_R4) for df in theopen_leaderboards_dict.values()]):.1f}")

Mean round score IQR in R1 for top half = 3.1
Mean round score IQR in R2 for top half = 3.6
Mean round score IQR in R3 for top half = 5.2
Mean round score IQR in R4 for top half = 6.8


Other ideas: 
- scores in R3, R4 reverting to mean; i.e. are those at bottom of top half likely to score better than their mean so far, moreso than leaders?

Plots to do:

- 3 round changes, boxplot etc of mean abs pos change for tournament, colour/facet by major
- 3 round changes, boxplot etc of K-T for rankings, colour/facet by major
- something to show total and/or round scores IQRs in each round
- boxplot of round scores

## US Masters

Historic leaderboards only include those that made the cut.

In [5]:
# response = br.open('https://www.augusta.com/masters/historic/leaderboards/2010')
# #response = br.open('https://www.augusta.com/masters/leaderboard')
# pagetext = response.readlines()
# pagetext = [b.decode() for b in pagetext]
# pagetext = [l.strip() for l in pagetext]

In [117]:
masters_leaderboards_dict = {}

for year in range(1990,2021):
    print(year)
    response = br.open('https://www.augusta.com/masters/historic/leaderboards/'+str(year))
    pagetext = response.readlines()
    pagetext = [b.decode() for b in pagetext]
    pagetext = [l.strip() for l in pagetext]

    lboard_started = False
    leaderboard = []
    for i,row in enumerate(pagetext):
        if 'id="LargeLeaderboard"' in row:
            #print(row)
            lboard_started = True
        if lboard_started:
            if '<tr' in row:
                scores = [l for l in pagetext[i+12:i+25] if re.match('\d{2}', l) and len(l) <= 4]
                if len(scores) == 4:
                    leaderboard.append({'player': pagetext[i+6], 'R1': int(scores[0]), 'R2': int(scores[1]), 'R3': int(scores[2]), 'R4': int(scores[3])})

    leaderboard_df = pd.DataFrame(leaderboard)
    leaderboard_df['made_cut'] = True
    leaderboard_df['total_at_R1'] = leaderboard_df['R1']
    leaderboard_df['total_at_R2'] = leaderboard_df['R1'] + leaderboard_df['R2']
    leaderboard_df['total_at_R3'] = leaderboard_df['total_at_R2'] + leaderboard_df['R3']
    leaderboard_df['total_at_R4'] = leaderboard_df['total_at_R3'] + leaderboard_df['R4']
    leaderboard_df['pos_at_R1'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R1 < row['total_at_R1']) + 1, axis=1)
    leaderboard_df['pos_at_R2'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R2 < row['total_at_R2']) + 1, axis=1)
    leaderboard_df['pos_at_R3'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R3 < row['total_at_R3']) + 1, axis=1)
    leaderboard_df['pos_at_R4'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R4 < row['total_at_R4']) + 1, axis=1)

    masters_leaderboards_dict[year] = leaderboard_df


1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [118]:
#Add 2021; this also only gets those making the cut

response = br.open('https://www.augusta.com/masters/leaderboard')
pagetext = response.readlines()
pagetext = [b.decode() for b in pagetext]
pagetext = [l.strip() for l in pagetext]

lboard_started = False
leaderboard = []
for i,row in enumerate(pagetext):
    if 'id="LargeLeaderboard"' in row:
        lboard_started = True
    if lboard_started:
        if re.match('<td>[T\d]*</td>', row):
            player_name = re.search('(.*) <', pagetext[i+4]).groups()[0]
            scores = [l for l in pagetext[i+4:i+15] if re.search('RoundData">\d{2}<', l)]
            scores = [re.search('RoundData">(\d{2})<', l).groups()[0] for l in scores]
            if len(scores) == 4:
                leaderboard.append({'player': player_name, 'R1': int(scores[0]), 'R2': int(scores[1]), 'R3': int(scores[2]), 'R4': int(scores[3])})

leaderboard_df = pd.DataFrame(leaderboard)
leaderboard_df['made_cut'] = True
leaderboard_df['total_at_R1'] = leaderboard_df['R1']
leaderboard_df['total_at_R2'] = leaderboard_df['R1'] + leaderboard_df['R2']
leaderboard_df['total_at_R3'] = leaderboard_df['total_at_R2'] + leaderboard_df['R3']
leaderboard_df['total_at_R4'] = leaderboard_df['total_at_R3'] + leaderboard_df['R4']
leaderboard_df['pos_at_R1'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R1 < row['total_at_R1']) + 1, axis=1)
leaderboard_df['pos_at_R2'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R2 < row['total_at_R2']) + 1, axis=1)
leaderboard_df['pos_at_R3'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R3 < row['total_at_R3']) + 1, axis=1)
leaderboard_df['pos_at_R4'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R4 < row['total_at_R4']) + 1, axis=1)

masters_leaderboards_dict[2021] = leaderboard_df


In [ ]:
mapc_r12, mapc_r23, mapc_r34 = [], [], []

for year, this_l_df in masters_leaderboards_dict.items():
    #print('\n'+str(year)+'\n')
    
    #Compare two ordered lists
    order_at_R1 = this_l_df.sort_values('total_at_R1').player.tolist()
    order_at_R2 = this_l_df.sort_values('total_at_R2').player.tolist()
    order_at_R1_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R1').player.tolist()
    order_at_R2_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R2').player.tolist()
    order_at_R3_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R3').player.tolist()
    order_at_R4_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R4').player.tolist()
    #print(f"{len(order_at_R1)} in R1 list, {len(order_at_R2)} in R2 list, {len(order_at_R2_tophalf)} in R2 cut only list, {len(order_at_R3_tophalf)} in R3 list, {len(order_at_R4_tophalf)} in R4 list")
    
    mapc_r12.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R1 - this_l_df[this_l_df.made_cut].pos_at_R2)))
    #print(f"Mean abs pos change R1->R2 for top half = {mapc_r12[-1]:.1f}")
    mapc_r23.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R2 - this_l_df[this_l_df.made_cut].pos_at_R3)))
    #print(f"Mean abs pos change R2->R3 for top half = {mapc_r23[-1]:.1f}")
    mapc_r34.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R3 - this_l_df[this_l_df.made_cut].pos_at_R4)))
    #print(f"Mean abs pos change R3->R4 for top half = {mapc_r34[-1]:.1f}")
    
print(f"Mean abs pos changes for top half are {np.mean(mapc_r12):.1f}, {np.mean(mapc_r23):.1f}, {np.mean(mapc_r34):.1f}")


In [147]:
#Round scores are basically the same

print(f"Mean 10% score in R1 for top half = {np.mean([df[df.made_cut].R1.quantile(0.10) for df in masters_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R2 for top half = {np.mean([df[df.made_cut].R2.quantile(0.10) for df in masters_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R3 for top half = {np.mean([df[df.made_cut].R3.quantile(0.10) for df in masters_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R4 for top half = {np.mean([df[df.made_cut].R4.quantile(0.10) for df in masters_leaderboards_dict.values()]):.1f}")

Mean 10% score in R1 for top half = 69.2
Mean 10% score in R2 for top half = 68.8
Mean 10% score in R3 for top half = 69.3
Mean 10% score in R4 for top half = 69.0


In [120]:
#As in Open, IQR seems highest in R4, then R3 - more going for broke?

print(f"Mean round score IQR in R1 for top half = {np.mean([iqr(df[df.made_cut].R1) for df in masters_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R2 for top half = {np.mean([iqr(df[df.made_cut].R2) for df in masters_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R3 for top half = {np.mean([iqr(df[df.made_cut].R3) for df in masters_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R4 for top half = {np.mean([iqr(df[df.made_cut].R4) for df in masters_leaderboards_dict.values()]):.1f}")

Mean round score IQR in R1 for top half = 3.2
Mean round score IQR in R2 for top half = 3.2
Mean round score IQR in R3 for top half = 3.7
Mean round score IQR in R4 for top half = 4.0


In [121]:
#As in Open, total score IQR increases by round, so it will be harder to move positions.

print(f"Mean round score IQR in R1 for top half = {np.mean([iqr(df[df.made_cut].total_at_R1) for df in masters_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R2 for top half = {np.mean([iqr(df[df.made_cut].total_at_R2) for df in masters_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R3 for top half = {np.mean([iqr(df[df.made_cut].total_at_R3) for df in masters_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R4 for top half = {np.mean([iqr(df[df.made_cut].total_at_R4) for df in masters_leaderboards_dict.values()]):.1f}")

Mean round score IQR in R1 for top half = 3.2
Mean round score IQR in R2 for top half = 4.0
Mean round score IQR in R3 for top half = 5.5
Mean round score IQR in R4 for top half = 7.3


## US Open

Use selenium for europeantour.com (slower); tables are blank before 1999.

In [122]:
usopen_leaderboards_dict = {}

for year in range(1999,2022):
    if year == 2020:
        driver.get(f'https://www.europeantour.com/european-tour/us-open-{year}/leaderboard')
    else:
        driver.get(f'https://www.europeantour.com/european-tour/u-s-open-{year}/leaderboard')
    time.sleep(2)
    lboard_rows = driver.find_elements_by_class_name('table-responsive')[0].find_elements_by_class_name('leaderboard__row')
    print(year, len(lboard_rows))

    leaderboard = []
    for row in lboard_rows:
        #print(row.find_elements_by_class_name('table__cell--name-col.text--left')[0].text)
        #print([row.find_elements_by_class_name('table__cell--grey.table__cell--round-strokes.table__cell--round-'+str(r))[0].text for r in [1,2,3,4]])

        player_name = row.find_elements_by_class_name('table__cell--name-col.text--left')[0].text
        player_name = player_name.split('\n')[0] #some have a logo after the name
        round_scores = [row.find_elements_by_class_name('table__cell--grey.table__cell--round-strokes.table__cell--round-'+str(r))[0].text for r in [1,2,3,4]]
        leaderboard.append({'player': player_name, 
                            'R1': None if round_scores[0]=='-' else int(round_scores[0]), 
                            'R2': None if round_scores[1]=='-' else int(round_scores[1]), 
                            'R3': None if round_scores[2]=='-' else int(round_scores[2]), 
                            'R4': None if round_scores[3]=='-' else int(round_scores[3])})

    leaderboard_df = pd.DataFrame(leaderboard)
    leaderboard_df['made_cut'] = ~leaderboard_df.R3.isnull()
    leaderboard_df['total_at_R1'] = leaderboard_df['R1']
    leaderboard_df['total_at_R2'] = leaderboard_df['R1'] + leaderboard_df['R2']
    leaderboard_df['total_at_R3'] = leaderboard_df['total_at_R2'] + leaderboard_df['R3']
    leaderboard_df['total_at_R4'] = leaderboard_df['total_at_R3'] + leaderboard_df['R4']
    leaderboard_df['pos_at_R1'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R1 < row['total_at_R1']) + 1, axis=1)
    leaderboard_df['pos_at_R2'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R2 < row['total_at_R2']) + 1, axis=1)
    leaderboard_df['pos_at_R3'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R3 < row['total_at_R3']) + 1, axis=1)
    leaderboard_df['pos_at_R4'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R4 < row['total_at_R4']) + 1, axis=1)

    usopen_leaderboards_dict[year] = leaderboard_df

1999 155
2000 156
2001 155
2002 155
2003 155
2004 169
2005 209
2006 202
2007 207
2008 155
2009 238
2010 227
2011 228
2012 231
2013 257
2014 156
2015 156
2016 156
2017 156
2018 156
2019 156
2020 144
2021 155


In [ ]:
mapc_r12, mapc_r23, mapc_r34 = [], [], []

for year, this_l_df in usopen_leaderboards_dict.items():
    order_at_R1 = this_l_df.sort_values('total_at_R1').player.tolist()
    order_at_R2 = this_l_df.sort_values('total_at_R2').player.tolist()
    order_at_R1_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R1').player.tolist()
    order_at_R2_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R2').player.tolist()
    order_at_R3_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R3').player.tolist()
    order_at_R4_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R4').player.tolist()
        
    mapc_r12.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R1 - this_l_df[this_l_df.made_cut].pos_at_R2)))
    mapc_r23.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R2 - this_l_df[this_l_df.made_cut].pos_at_R3)))
    mapc_r34.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R3 - this_l_df[this_l_df.made_cut].pos_at_R4)))
    
print(f"Mean abs pos changes for top half are {np.mean(mapc_r12):.1f}, {np.mean(mapc_r23):.1f}, {np.mean(mapc_r34):.1f}")


In [148]:
#Round scores are slightly worse in R3-4

print(f"Mean 10% score in R1 for top half = {np.mean([df[df.made_cut].R1.quantile(0.10) for df in usopen_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R2 for top half = {np.mean([df[df.made_cut].R2.quantile(0.10) for df in usopen_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R3 for top half = {np.mean([df[df.made_cut].R3.quantile(0.10) for df in usopen_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R4 for top half = {np.mean([df[df.made_cut].R4.quantile(0.10) for df in usopen_leaderboards_dict.values()]):.1f}")

Mean 10% score in R1 for top half = 69.2
Mean 10% score in R2 for top half = 69.0
Mean 10% score in R3 for top half = 69.7
Mean 10% score in R4 for top half = 69.9


In [127]:
#As previous, IQR seems highest in R4, then R3 - more going for broke?

print(f"Mean round score IQR in R1 for top half = {np.mean([iqr(df[df.made_cut].R1) for df in usopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R2 for top half = {np.mean([iqr(df[df.made_cut].R2) for df in usopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R3 for top half = {np.mean([iqr(df[df.made_cut].R3) for df in usopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R4 for top half = {np.nanmean([iqr(df[df.made_cut].R4) for df in usopen_leaderboards_dict.values()]):.1f}")

Mean round score IQR in R1 for top half = 3.1
Mean round score IQR in R2 for top half = 3.1
Mean round score IQR in R3 for top half = 4.0
Mean round score IQR in R4 for top half = 4.0


In [126]:
#As previous, total score IQR increases by round, so it will be harder to move positions.

print(f"Mean round score IQR in R1 for top half = {np.mean([iqr(df[df.made_cut].total_at_R1) for df in usopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R2 for top half = {np.mean([iqr(df[df.made_cut].total_at_R2) for df in usopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R3 for top half = {np.mean([iqr(df[df.made_cut].total_at_R3) for df in usopen_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R4 for top half = {np.nanmean([iqr(df[df.made_cut].total_at_R4) for df in usopen_leaderboards_dict.values()]):.1f}")

Mean round score IQR in R1 for top half = 3.1
Mean round score IQR in R2 for top half = 3.7
Mean round score IQR in R3 for top half = 5.5
Mean round score IQR in R4 for top half = 7.1


## US PGA

As previous.

In [154]:
uspga_leaderboards_dict = {}

for year in range(1999,2022):
    if year == 2021:
        driver.get(f'https://www.europeantour.com/european-tour/u-s-pga-championship-{year}/leaderboard')
    else:
        driver.get(f'https://www.europeantour.com/european-tour/us-pga-championship-{year}/leaderboard')
    time.sleep(2)
    lboard_rows = driver.find_elements_by_class_name('table-responsive')[0].find_elements_by_class_name('leaderboard__row')

    leaderboard = []
    for row in lboard_rows:
        player_name = row.find_elements_by_class_name('table__cell--name-col.text--left')[0].text
        player_name = player_name.split('\n')[0] #some have a logo after the name
        round_scores = [row.find_elements_by_class_name('table__cell--grey.table__cell--round-strokes.table__cell--round-'+str(r))[0].text for r in [1,2,3,4]]
        leaderboard.append({'player': player_name, 
                            'R1': None if round_scores[0]=='-' else int(round_scores[0]), 
                            'R2': None if round_scores[1]=='-' else int(round_scores[1]), 
                            'R3': None if round_scores[2]=='-' else int(round_scores[2]), 
                            'R4': None if round_scores[3]=='-' else int(round_scores[3])})
    print(year, len(lboard_rows), len(leaderboard))

    leaderboard_df = pd.DataFrame(leaderboard)
    leaderboard_df['made_cut'] = ~leaderboard_df.R3.isnull()
    leaderboard_df['total_at_R1'] = leaderboard_df['R1']
    leaderboard_df['total_at_R2'] = leaderboard_df['R1'] + leaderboard_df['R2']
    leaderboard_df['total_at_R3'] = leaderboard_df['total_at_R2'] + leaderboard_df['R3']
    leaderboard_df['total_at_R4'] = leaderboard_df['total_at_R3'] + leaderboard_df['R4']
    leaderboard_df['pos_at_R1'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R1 < row['total_at_R1']) + 1, axis=1)
    leaderboard_df['pos_at_R2'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R2 < row['total_at_R2']) + 1, axis=1)
    leaderboard_df['pos_at_R3'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R3 < row['total_at_R3']) + 1, axis=1)
    leaderboard_df['pos_at_R4'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R4 < row['total_at_R4']) + 1, axis=1)

    uspga_leaderboards_dict[year] = leaderboard_df

1999 149
149
2000 150
150
2001 150
150
2002 155
155
2003 155
155
2004 156
156
2005 156
156
2006 156
156
2007 156
156
2008 156
156
2009 156
156
2010 156
156
2011 156
156
2012 156
156
2013 156
156
2014 156
156
2015 156
156
2016 156
156
2017 156
156
2018 156
156
2019 156
156
2020 156
156
2021 156
156


In [ ]:
mapc_r12, mapc_r23, mapc_r34 = [], [], []

for year, this_l_df in uspga_leaderboards_dict.items():
    order_at_R1 = this_l_df.sort_values('total_at_R1').player.tolist()
    order_at_R2 = this_l_df.sort_values('total_at_R2').player.tolist()
    order_at_R1_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R1').player.tolist()
    order_at_R2_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R2').player.tolist()
    order_at_R3_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R3').player.tolist()
    order_at_R4_tophalf = this_l_df[this_l_df.made_cut].sort_values('total_at_R4').player.tolist()
        
    mapc_r12.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R1 - this_l_df[this_l_df.made_cut].pos_at_R2)))
    mapc_r23.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R2 - this_l_df[this_l_df.made_cut].pos_at_R3)))
    mapc_r34.append(np.mean(abs(this_l_df[this_l_df.made_cut].pos_at_R3 - this_l_df[this_l_df.made_cut].pos_at_R4)))
    
print(f"Mean abs pos changes for top half are {np.mean(mapc_r12):.1f}, {np.mean(mapc_r23):.1f}, {np.mean(mapc_r34):.1f}")


In [156]:
#Round scores are nearly identical

print(f"Mean 10% score in R1 for top half = {np.mean([df[df.made_cut].R1.quantile(0.10) for df in uspga_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R2 for top half = {np.mean([df[df.made_cut].R2.quantile(0.10) for df in uspga_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R3 for top half = {np.mean([df[df.made_cut].R3.quantile(0.10) for df in uspga_leaderboards_dict.values()]):.1f}")
print(f"Mean 10% score in R4 for top half = {np.mean([df[df.made_cut].R4.quantile(0.10) for df in uspga_leaderboards_dict.values()]):.1f}")

Mean 10% score in R1 for top half = 68.4
Mean 10% score in R2 for top half = 68.3
Mean 10% score in R3 for top half = 68.4
Mean 10% score in R4 for top half = 68.4


In [157]:
#As previous, IQR seems highest in R4, R3 - more going for broke?

print(f"Mean round score IQR in R1 for top half = {np.mean([iqr(df[df.made_cut].R1) for df in uspga_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R2 for top half = {np.mean([iqr(df[df.made_cut].R2) for df in uspga_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R3 for top half = {np.mean([iqr(df[df.made_cut].R3) for df in uspga_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R4 for top half = {np.nanmean([iqr(df[df.made_cut].R4) for df in uspga_leaderboards_dict.values()]):.1f}")

Mean round score IQR in R1 for top half = 3.0
Mean round score IQR in R2 for top half = 3.0
Mean round score IQR in R3 for top half = 3.8
Mean round score IQR in R4 for top half = 3.8


In [158]:
#As previous, total score IQR increases by round, so it will be harder to move positions.

print(f"Mean round score IQR in R1 for top half = {np.mean([iqr(df[df.made_cut].total_at_R1) for df in uspga_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R2 for top half = {np.mean([iqr(df[df.made_cut].total_at_R2) for df in uspga_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R3 for top half = {np.mean([iqr(df[df.made_cut].total_at_R3) for df in uspga_leaderboards_dict.values()]):.1f}")
print(f"Mean round score IQR in R4 for top half = {np.nanmean([iqr(df[df.made_cut].total_at_R4) for df in uspga_leaderboards_dict.values()]):.1f}")

Mean round score IQR in R1 for top half = 3.0
Mean round score IQR in R2 for top half = 3.5
Mean round score IQR in R3 for top half = 5.2
Mean round score IQR in R4 for top half = 7.2


## Save everything to a file

In [159]:
out_dfs = []
for year, tmp in theopen_leaderboards_dict.items():
    tmp['year'] = year
    tmp['comp'] = 'theopen'
    out_dfs.append(tmp)

In [160]:
for year, tmp in masters_leaderboards_dict.items():
    tmp['year'] = year
    tmp['comp'] = 'masters'
    out_dfs.append(tmp)

In [162]:
for year, tmp in uspga_leaderboards_dict.items():
    tmp['year'] = year
    tmp['comp'] = 'uspga'
    out_dfs.append(tmp)

In [163]:
for year, tmp in usopen_leaderboards_dict.items():
    tmp['year'] = year
    tmp['comp'] = 'usopen'
    out_dfs.append(tmp)

In [171]:
pd.concat(out_dfs).comp.value_counts()
#masters only shows making cut

usopen     4134
uspga      3567
theopen    3382
masters    1682
Name: comp, dtype: int64

In [ ]:
pd.concat(out_dfs).to_csv('golf_majors_round_data_1990_2021.csv', index=False)

### Get Masters from europeantour.com instead

It is partially populated before 1999

In [4]:
driver = webdriver.Firefox()

In [15]:
masters_b_leaderboards_dict = {}

for year in range(1990,2022):
    if year >= 2018:
        driver.get(f'https://www.europeantour.com/european-tour/the-masters-{year}/results?round=4')
    else:
        driver.get(f'https://www.europeantour.com/european-tour/masters-tournament-{year}/results?round=4')
    time.sleep(2)
    lboard_rows = driver.find_elements_by_class_name('table-responsive')[0].find_elements_by_class_name('leaderboard__row')

    leaderboard = []
    for row in lboard_rows:
        player_name = row.find_elements_by_class_name('table__cell--name-col.text--left')[0].text
        player_name = player_name.split('\n')[0] #some have a logo after the name
        round_scores = [row.find_elements_by_class_name('table__cell--grey.table__cell--round-strokes.table__cell--round-'+str(r))[0].text for r in [1,2,3,4]]
        leaderboard.append({'player': player_name, 
                            'R1': None if round_scores[0]=='-' else int(round_scores[0]), 
                            'R2': None if round_scores[1]=='-' else int(round_scores[1]), 
                            'R3': None if round_scores[2]=='-' else int(round_scores[2]), 
                            'R4': None if round_scores[3]=='-' else int(round_scores[3])})
    print(year, len(lboard_rows), len(leaderboard))

    leaderboard_df = pd.DataFrame(leaderboard)
    leaderboard_df['made_cut'] = ~leaderboard_df.R3.isnull()
    leaderboard_df['total_at_R1'] = leaderboard_df['R1']
    leaderboard_df['total_at_R2'] = leaderboard_df['R1'] + leaderboard_df['R2']
    leaderboard_df['total_at_R3'] = leaderboard_df['total_at_R2'] + leaderboard_df['R3']
    leaderboard_df['total_at_R4'] = leaderboard_df['total_at_R3'] + leaderboard_df['R4']
    leaderboard_df['pos_at_R1'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R1 < row['total_at_R1']) + 1, axis=1)
    leaderboard_df['pos_at_R2'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R2 < row['total_at_R2']) + 1, axis=1)
    leaderboard_df['pos_at_R3'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R3 < row['total_at_R3']) + 1, axis=1)
    leaderboard_df['pos_at_R4'] = leaderboard_df.apply(lambda row: sum(leaderboard_df.total_at_R4 < row['total_at_R4']) + 1, axis=1)

    masters_b_leaderboards_dict[year] = leaderboard_df

1990 85 85
1991 87 87
1992 83 83
1993 90 90
1994 86 86
1995 86 86
1996 92 92
1997 87 87
1998 90 90
1999 97 97
2000 95 95
2001 93 93
2002 88 88
2003 93 93
2004 93 93
2005 93 93
2006 90 90
2007 96 96
2008 95 95
2009 96 96
2010 97 97
2011 99 99
2012 95 95
2013 93 93
2014 97 97
2015 97 97
2016 89 89
2017 93 93
2018 87 87
2019 87 87
2020 92 92
2021 88 88


In [18]:
masters_b_leaderboards_dict[2021]

,player,R1,R2,R3,R4,made_cut,total_at_R1,total_at_R2,total_at_R3,total_at_R4,pos_at_R1,pos_at_R2,pos_at_R3,pos_at_R4
0,"MATSUYAMA, Hideki",69,71.0,65.0,73.0,True,69,140.0,205.0,278.0,2,6,1,1
1,"ZALATORIS, Will",70,68.0,71.0,70.0,True,70,138.0,209.0,279.0,4,2,2,2
2,"SCHAUFFELE, Xander",72,69.0,68.0,72.0,True,72,141.0,209.0,281.0,13,12,2,3
3,"SPIETH, Jordan",71,68.0,72.0,70.0,True,71,139.0,211.0,281.0,8,4,7,3
4,"LEISHMAN, Marc",72,67.0,70.0,73.0,True,72,139.0,209.0,282.0,13,4,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,"IM, Sungjae",77,80.0,NaN,NaN,False,77,157.0,NaN,NaN,72,83,1,1
84,"SINGH, Vijay",79,80.0,NaN,NaN,False,79,159.0,NaN,NaN,80,85,1,1
85,"STRAFACI, Tyler",80,81.0,NaN,NaN,False,80,161.0,NaN,NaN,84,86,1,1
86,"MIZE, Larry",84,79.0,NaN,NaN,False,84,163.0,NaN,NaN,88,87,1,1


In [29]:
masters_replace_df = []
for year, tmp in masters_b_leaderboards_dict.items():
    tmp['year'] = year
    tmp['comp'] = 'masters'
    masters_replace_df.append(tmp)
masters_replace_df = pd.concat(masters_replace_df)    

In [30]:
masters_replace_df.shape

(2929, 16)

In [31]:
df_old = pd.read_csv('golf_majors_round_data_1990_2021.csv')
print(df_old.shape)
df_new = pd.concat([df_old[df_old.comp != 'masters'], masters_replace_df])
df_new.shape

(12765, 16)


(14012, 16)

In [32]:
df_new.comp.value_counts()

usopen     4134
uspga      3567
theopen    3382
masters    2929
Name: comp, dtype: int64

In [33]:
df_new.to_csv('golf_majors_round_data_1990_2021.csv', index=False)